In [1]:
import glob, os
import pandas as pd
import pdb
import subprocess
from IPython.core import display as ICD
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)
import numpy as np

In [2]:
bids_path = '/projects/niblab/bids_projects/Experiments/bbx/bids/'
scan_file ="bbx_scan_11.19.tsv"
report = pd.read_csv(scan_file,sep="\t")

In [3]:
report.columns = ['subject', 'BIDS check', 'fmriprep check', 'Anat', 'epi_r1',
       'sdc_r1', 'epi_r2', 'sdc_r2', 'epi_r3', 'sdc_r3',
       'epi_r4', 'sdc_r4', 'From TSV Files', 'fd_r1', 'min_r1',
       'max_r1', 'mcflirt_3mm_r1', 'fd_r2', 'min_r2',
       'max_r2', 'mcflirt_3mm_r2', 'fd_r3', 'min_r3',
       'max_r3', 'mcflirt_3mm_r3', 'fd_r4', 'min_r4',
       'max_r4', 'mcflirt_3mm_r4', 'FSL Level 1', 'activation_r1', 'activation_r2',
       'activation_r3', 'activation_r4', 'FINAL CHECK', 'Intials']


col_details = report.iloc[0:2, :]
col_details['fd_r1']
report.drop([0,1], inplace=True)
report = report.set_index(["subject"])


In [4]:
# get scan notes
df_w1_notes=pd.read_csv('w1_notes.csv', encoding='latin-1')
#df_w1_notes.head()
#df_w1_notes.columns
df_clean=df_w1_notes[['participantID', 'w1scan_scannotes']]

df_clean.set_index("participantID", inplace=True)
df_clean.index = df_clean.index.str.lower()
df_clean = df_clean.drop(['participant id (bbx_###)'])

## DICOM Check

In [5]:

    
# Get DICOM id list
s1_dcms = [x.split("/")[-1].split("-")[1].lstrip("0") for x in
             glob.glob(os.path.join(bids_path, "sourcedata/DICOM/ses-1/sub-*"))]
s1_dcms = np.unique(np.array(s1_dcms)).tolist()
# get expected id list from notes
s1_sub_ids = [x.split("_")[1].lstrip('0') for x in df_clean.index.values]
s1_sub_ids = np.unique(np.array(s1_sub_ids)).tolist()

# get total count for DICOM and expected id lists
s1_sub_exp_ct = len(s1_sub_ids)
s1_sub_dcm_ct = len(s1_dcms)

# return the unique values in ar1 that are not in ar2
# ids found in dicom directory but not id list
s1_mia_id = np.setdiff1d(s1_dcms, s1_sub_ids)

# ids missing from dicom directories
s1_mia_dcm = np.setdiff1d(s1_sub_ids, s1_dcms)

s1_mia_id = s1_mia_id.tolist()
s1_mia_dcm = s1_mia_dcm.tolist()

# look at scan notes for any missing DICOM ids
for id_ in s1_mia_dcm:
    bbx_id = "bbx_{:03d}".format(int(id_))
    try:
        print("{}, notes: \n{}".format(bbx_id, df_clean.loc[bbx_id]))
    except:
        print("Missing scan notes for ", bbx_id)
        


## Report Output
print("BBx Session 1 DICOM info:")
print("\nScan notes subject count: {} \tUnique DICOM directories found: {}".format(s1_sub_exp_ct, s1_sub_dcm_ct))
print("\n>>> Missing DICOM directories for IDs: {} \n\n>>> Missing scan notes for IDs: {} \n".format(s1_mia_dcm, s1_mia_id))
print("DICOM IDs available: \t",sorted(list(map(int, s1_sub_ids ))))



bbx_128, notes: 
w1scan_scannotes    Some movement throughout but not too bad 
Name: bbx_128, dtype: object
bbx_094, notes: 
w1scan_scannotes    Consistently moderate twitching motion throughout due to swallowing 
Name: bbx_094, dtype: object
BBx Session 1 DICOM info:

Scan notes subject count: 142 	Unique DICOM directories found: 141

>>> Missing DICOM directories for IDs: ['128', '94'] 

>>> Missing scan notes for IDs: ['154'] 

DICOM IDs available: 	 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102, 103, 104, 107, 108, 109, 111, 112, 114, 115, 116, 117, 118, 119, 120, 121, 123, 124, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 140, 141, 142, 143,

## BIDS Check  
Other:    
* BIDS validator  
* Volume Check
#### Compare DICOM ids with BIDS ids found, list any missing from BIDS

In [6]:
s1_bids = np.unique(np.array([x.split("/")[-2].split("-")[1].lstrip("0") for x in
             glob.glob(os.path.join(bids_path, "sub-*/ses-1"))])).tolist()
s1_bids_ct = len(s1_bids)

# subjects found in DICOM list but not in bids
s1_mia_bids = np.setdiff1d(s1_dcms, s1_bids)

s1_bids = sorted(list(map(int,  s1_bids)))
# report
print("BIDS Session 1 Info: ")
print("Unique BIDS subject count: {} \n\n>>> Missing IDs: \t{} \n".format(s1_bids_ct, s1_mia_bids))
print("BIDS IDs found: ", s1_bids)

BIDS Session 1 Info: 
Unique BIDS subject count: 141 

>>> Missing IDs: 	[] 

BIDS IDs found:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 102, 103, 104, 107, 108, 109, 111, 112, 114, 115, 116, 117, 118, 119, 120, 121, 123, 124, 127, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 156, 157, 159]


#### Fill in a dictionary with subject ids and find their directories:  
#### *Value of 1 translates that the directory and files are missing*

In [7]:

# if no missing IDs found, that is good, meaning we have at least attempted to generate all of the subject directories available
# now we have to inspect individual directories
qa_dict={}
bids_dirs = glob.glob(os.path.join(bids_path, "sub-*/ses-1"))
for dir_path in sorted(bids_dirs):
    sub_id = dir_path.split("/")[-2]
    if sub_id not in qa_dict:
        qa_dict[sub_id] = {
            "FUNC": 0,
            "FMAP": 0,
            "ANAT": 0
        }
        
    funcs=glob.glob(os.path.join(dir_path, "func/*"))
    anat=glob.glob(os.path.join(dir_path, "fmap/*"))
    fmaps=glob.glob(os.path.join(dir_path, "anat/*"))
    #print("Analysis of {}:".format(sub_id))
    if not funcs:
        qa_dict[sub_id]["FUNC"] = 1
    if not fmaps:
        qa_dict[sub_id]["FMAP"] = 1
    if not anat:
        qa_dict[sub_id]["ANAT"] = 1
    
    
    

In [8]:
bids_df = pd.DataFrame(qa_dict).T

#### Now we need to pull out any subjects that have missing files:

In [9]:
missing_df = bids_df[bids_df.any(axis="columns")]

In [10]:
ICD.display(missing_df)

,FUNC,FMAP,ANAT


If our dataframe is empty all of our subjects have generated complete BIDS structures,  
else subjects are listed, where 1 represents missing data.

In [11]:
bids_ids_mia=list(missing_df.index.values)

#### Now lets try to get as much information about the missing subjects as we can:   



In [12]:

for sub_id in missing_df.index.values:
    print("\nID: ",sub_id)
    dir_path = glob.glob(os.path.join(bids_path, sub_id, "ses-1/*"))
    print("\nFiles Found: ")
    for z in dir_path:
        print(z)
    try:
        print("\nDIRECTORIES FOUND MISSING: ")
        ICD.display( missing_df.loc[sub_id])
        print("SCAN NOTES (IF AVAILABLE):")
        ICD.display( df_clean.loc[sub_id.replace("sub-", "bbx_")])
    except:
        print("\nNo scan notes found")
    print("____________________________________")

In [13]:
bids_complete_ct=len(s1_bids)-len(bids_ids_mia)
expected_ids = set(s1_bids) - set([int(x.split('-')[1].lstrip('0')) for x in bids_ids_mia])
expected_ids=list(expected_ids)
print("BIDS complete subject count: ", bids_complete_ct)
print((expected_ids))

BIDS complete subject count:  141
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 102, 103, 104, 107, 108, 109, 111, 112, 114, 115, 116, 117, 118, 119, 120, 121, 123, 124, 127, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 156, 157, 159]


In [14]:
print("missing ids: \t", " ".join(bids_ids_mia))

missing ids: 	 


In [15]:
print(" ".join([x.split("-")[1].lstrip('0') for x in bids_ids_mia]))

## Now we Check Volume:

In [16]:
for x in expected_ids:
    sub_id = "sub-{:03d}".format(x)
    funcs=glob.glob(os.path.join(bids_path,sub_id, "ses-1/func/*.nii.gz" ))
    for func in funcs:
        filename = func.split("/")[-1]
        fsl_cmd ="fslnvols {}".format(func)
        vol=subprocess.check_output(fsl_cmd, shell=True)
        vol=str(vol,'utf-8').strip()
        if "training" in func:
            expected_vol = 233
        elif "rl" in func:
            expected_vol = 212
        else:
            expected_vol = 147
        if int(vol) != expected_vol:
            if "training" in func:
                line="File:{} \t Volume:{}".format(filename, vol)
            else:
                line="File:{} \t\t Volume:{}".format(filename, vol)
            print(line)

File:sub-159_ses-1_task-resting_bold.nii.gz 		 Volume:68


In [17]:
#expected_ids.remove(94)
#expected_ids.remove(128)

print("BIDS Subject Count: ", len(expected_ids))
print("IDs: \t",expected_ids)

BIDS Subject Count:  141
IDs: 	 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 102, 103, 104, 107, 108, 109, 111, 112, 114, 115, 116, 117, 118, 119, 120, 121, 123, 124, 127, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 156, 157, 159]


## Notes:  
* sub-094 and sub-128 each had unqiue identifiers for heuristic file and have no functional runs with correct volumes, will drop subjects
* sub-155, *dropped before scans*  

In [18]:
report_tsv = "bbx_scan_11.19.tsv"
report_df=pd.read_csv(scan_file,sep="\t")
report_df.columns = ['subject', 'BIDS check', 'fmriprep check', 'Anat', 'epi_r1',
       'sdc_r1', 'epi_r2', 'sdc_r2', 'epi_r3', 'sdc_r3',
       'epi_r4', 'sdc_r4', 'From TSV Files', 'fd_r1', 'min_r1',
       'max_r1', 'mcflirt_3mm_r1', 'fd_r2', 'min_r2',
       'max_r2', 'mcflirt_3mm_r2', 'fd_r3', 'min_r3',
       'max_r3', 'mcflirt_3mm_r3', 'fd_r4', 'min_r4',
       'max_r4', 'mcflirt_3mm_r4', 'FSL Level 1', 'activation_r1', 'activation_r2',
       'activation_r3', 'activation_r4', 'FINAL CHECK', 'Intials']

col_details = report_df.iloc[0:2, :]
col_details['fd_r1']
report_df.drop([0,1], inplace=True)
report_df = report_df.set_index(["subject"])


In [19]:
report_df = report_df[:159]
report_df.head()

,BIDS check,fmriprep check,Anat,epi_r1,sdc_r1,epi_r2,sdc_r2,epi_r3,sdc_r3,epi_r4,...,min_r4,max_r4,mcflirt_3mm_r4,FSL Level 1,activation_r1,activation_r2,activation_r3,activation_r4,FINAL CHECK,Intials
subject,,,,,,,,,,,,,,,,,,,,,
sub-001,y,--,good,good,good,good,good,good,good,good,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-002,y,--,good,good,good,good,good,good,good,good,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-003,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-004,y,--,good,good,good,good,good,good,good,good,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-005,y,--,good,good,good,good,good,good,good,good,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
remove_ids = [index for index in report_df.index.values if int(index.split("-")[1]) not in expected_ids]
#for index in remove_ids

In [21]:
for index in remove_ids:
    try: 
        report_df.drop(index, inplace=True)
    except:
        pass
for x in expected_ids:
    index = "sub-{:03d}".format(x)
    funcs = glob.glob(os.path.join(bids_path, index, 'ses-1/func/*nii.gz'))
    if not funcs:
        print("MISSING FUNCS")
        pass
    else:
        report_df.loc[index, 'BIDS check'] = "y"


In [22]:
report_df

,BIDS check,fmriprep check,Anat,epi_r1,sdc_r1,epi_r2,sdc_r2,epi_r3,sdc_r3,epi_r4,...,min_r4,max_r4,mcflirt_3mm_r4,FSL Level 1,activation_r1,activation_r2,activation_r3,activation_r4,FINAL CHECK,Intials
subject,,,,,,,,,,,,,,,,,,,,,
sub-001,y,--,good,good,good,good,good,good,good,good,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-002,y,--,good,good,good,good,good,good,good,good,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-003,y,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-004,y,--,good,good,good,good,good,good,good,good,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-005,y,--,good,good,good,good,good,good,good,good,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-153,y,--,good,good,good,good,good,good,good,good,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-154,y,--,good,good,missing,good,missing,good,missing,good,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-156,y,--,good,good,missing,good,missing,good,missing,good,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
report_df.to_csv("bbx_qc_bids.tsv", sep="\t")